In [1]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import plotly.express as px
import plotly.graph_objects as go


import pandas as pd
import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.backtester.base.base import  plot_trades_all_info
from quantfreedom.backtester.nb.simulate import simulate_up_to_6
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.backtester.indicators.talib_ind import from_talib
from quantfreedom.backtester.evaluators.evaluators import eval_is_below

np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv(
    'E:/Coding/backtesters/QuantFreedom/tests/data/30min.csv', index_col='time')

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=15,
)

rsi_eval = eval_is_below(
    rsi_ind,
    np.arange(40, 61, 10),
)


In [2]:
np.where(
    rsi_eval.values,
    rsi_ind.values.flatten(),
    np.nan,
)

ValueError: operands could not be broadcast together with shapes (6960,3) (6960,) () 

In [2]:
final_array, order_records = simulate_up_to_6(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=rsi_eval.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    # sl_pcts=[2,3,4],
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2,5,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,4,1),
    tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [3]:
filtered_or = order_records[order_records['settings_id'] == 0]

In [14]:
d_table = pd.DataFrame(filtered_or)
for col in d_table:
    if d_table[col].dtype == 'float64':
        d_table[col] = d_table[col].map('{:,.2f}'.format)
d_table

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
0,0,0,31,471.97,"10,850.00","10,850.00",nan,0.00,nan,"1,000.00",nan,"10,633.00","11,553.11"
1,1,0,32,471.97,"10,725.00","10,787.14",nan,0.00,nan,"1,000.00",nan,"10,571.40","11,486.18"
2,2,0,32,943.93,"10,571.40","10,787.14",1.12,6.00,-20.00,980.00,nan,"10,571.40","11,486.18"
3,3,0,33,462.53,"10,577.50","10,577.50",nan,0.00,nan,980.00,nan,"10,365.95","11,262.95"
4,4,0,34,462.53,"10,523.00","10,550.18",nan,0.00,nan,980.00,nan,"10,339.18","11,233.86"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,1146,0,6952,nan,nan,"52,662.53",nan,12.00,nan,743.04,nan,"52,904.11","56,075.21"
1147,1147,0,6953,nan,nan,"52,662.53",nan,12.00,nan,743.04,nan,"53,229.82","56,075.21"
1148,1148,0,6955,nan,nan,"52,662.53",nan,12.00,nan,743.04,nan,"53,619.89","56,075.21"
1149,1149,0,6956,nan,nan,"52,662.53",nan,12.00,nan,743.04,nan,"53,758.49","56,075.21"


In [18]:
list(d_table.columns)

['order_id',
 'settings_id',
 'bar',
 'size_value',
 'price',
 'avg_entry',
 'fees_paid',
 'order_type',
 'real_pnl',
 'equity',
 'sl_prices',
 'tsl_prices',
 'tp_prices']

In [5]:
y = np.append(
        0,
        filtered_or["real_pnl"][
            ~np.isnan(filtered_or["real_pnl"])
        ].cumsum(),
    )

x = np.arange(0, y.size)

In [6]:
fig = go.Figure()

fig.add_scatter(x=x, y=y)
fig.show()

In [7]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.iloc[3009:3013]

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
3009,3009,2,4280,"1,996.75","26,413.58","26,336.85",2.40,LongTSL,3.42,"2,058.87",NaN,"26,413.58","32,879.06"
3010,3010,2,4281,500.02,"26,563.50","26,563.50",NaN,LongEntry,NaN,"2,058.87",NaN,"25,500.96","33,162.01"
3011,3011,2,4282,500.02,"26,504.50","26,533.97",NaN,LongEntry,NaN,"2,058.87",NaN,"25,472.61","33,125.14"
3012,3012,2,4283,500.02,"26,250.00","26,438.63",NaN,LongEntry,NaN,"2,058.87",NaN,"25,381.09","33,006.12"


In [8]:
df_or.iloc[1145:1155]

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
1145,1145,0,6951,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"52,675.43","56,075.21"
1146,1146,0,6952,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"52,904.11","56,075.21"
1147,1147,0,6953,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"53,229.82","56,075.21"
1148,1148,0,6955,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"53,619.89","56,075.21"
1149,1149,0,6956,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"53,758.49","56,075.21"
1150,1150,0,6958,"1,402.76","53,758.49","52,662.53",1.70,LongTSL,27.49,770.53,NaN,"53,758.49","56,075.21"
1151,1151,1,31,320.70,"10,850.00","10,850.00",NaN,LongEntry,NaN,"1,000.00",NaN,"10,524.50","12,555.67"
1152,1152,1,32,320.70,"10,725.00","10,787.14",NaN,LongEntry,NaN,"1,000.00",NaN,"10,463.52","12,482.92"
1153,1153,1,33,320.70,"10,577.50","10,716.34",NaN,LongEntry,NaN,"1,000.00",NaN,"10,394.85","12,401.00"
1154,1154,1,34,320.70,"10,523.00","10,667.34",NaN,LongEntry,NaN,"1,000.00",NaN,"10,347.32","12,344.30"


In [9]:
df_or.tail()

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
3678,3678,2,6941,658.17,"51,319.50","51,319.50",NaN,LongEntry,NaN,"2,710.10",NaN,"49,266.72","64,067.52"
3679,3679,2,6942,658.17,"53,278.00","52,280.41",NaN,LongEntry,NaN,"2,710.10",NaN,"50,189.20","65,267.13"
3680,3680,2,6943,658.17,"52,890.00","52,482.04",NaN,LongEntry,NaN,"2,710.10",NaN,"50,382.76","65,518.85"
3681,3681,2,6944,658.17,"53,211.00","52,662.40",NaN,LongEntry,NaN,"2,710.10",NaN,"50,555.91","65,744.01"
3682,3682,2,6956,NaN,NaN,"52,662.40",NaN,MovedTSL,NaN,"2,710.10",NaN,"52,672.46","65,744.01"


In [10]:
df_cart = pd.DataFrame(final_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2
total_trades,195.00,155.00,107.00
gains_pct,-22.95,57.47,171.01
win_rate,37.95,46.45,49.53
to_the_upside,0.67,0.11,0.27
total_pnl,-229.47,574.66,"1,710.10"
ending_eq,770.53,"1,574.66","2,710.10"
settings_id,0,1,2
max_equity_risk_pct,4.00,4.00,4.00
risk_rewards,3.00,5.00,6.00
size_pct,1.00,1.00,1.00


In [11]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[:30],
)

In [12]:
clear_cache()